In [1]:
import cv2
import numpy as np
import os


faces, labels = [], []
label_map = {"dan": 0}  # Example name-to-ID mapping

for name, label in label_map.items():
    for img_name in os.listdir(f"faces/{name}"):
        img = cv2.imread(f"faces/{name}/{img_name}", cv2.IMREAD_GRAYSCALE)
        faces.append(img)
        labels.append(label)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(labels))
recognizer.save("lbph_model.yml")



# === Load Trained LBPH Model ===
recognizer.read("lbph_model.yml")  # Make sure this file exists

# === Label Mapping ===
label_map_rev = {0: "dan"}  # Update with your labels

# === Load Haar Cascade for Face Detection ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# === Load Video ===
cap = cv2.VideoCapture(r'C:\Users\veyadav\Downloads\Drupal-Branching.mp4' )

# === Target Screen Size ===
screen_width, screen_height = 1280, 720

cv2.namedWindow("LBPH Face Matching", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("LBPH Face Matching", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        label, confidence = recognizer.predict(face_roi)
        name = label_map_rev.get(label, "Unknown")

        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"{name} ({confidence:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # === Resize Frame to Fit Screen ===
    fh, fw = frame.shape[:2]
    scale = min(screen_width / fw, screen_height / fh)
    new_w, new_h = int(fw * scale), int(fh * scale)
    resized_frame = cv2.resize(frame, (new_w, new_h))

    # Center on black canvas
    canvas = cv2.copyMakeBorder(
        resized_frame,
        top=(screen_height - new_h) // 2,
        bottom=(screen_height - new_h + 1) // 2,
        left=(screen_width - new_w) // 2,
        right=(screen_width - new_w + 1) // 2,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    cv2.imshow("LBPH Face Matching", canvas)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()